In [ ]:
from langchain_xai import ChatXAI
from langchain.vectorstores import FAISS
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.document_loaders import PyPDFLoader

In [ ]:
pdf_reder = PyPDFLoader('RAGPaper.pdf')
document = pdf_reder.load()

In [ ]:
text_spliter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)

chunk = text_spliter.split_documents(document)

In [ ]:
embedding = HuggingFaceEmbeddings()
db = FAISS.from_documents(documents=chunk, embedding=embedding)

In [ ]:
llm = ChatXAI(model='grok-2-latest',
               temperature=0.02,
               max_token=None,
               timeout=None,
               max_retries=2,
               api_key='API_KEY')

In [ ]:
from langchain.chains import ConversationalRetrievalChain
from langchain.prompts import PromptTemplate


CONDENSE_QUESTION_PROMPT = PromptTemplate.from_template("""Given The following conversation and a follow question, rephrase the follow up question to be a standalone question.
Chat History : {chat_history}
Follow up input : {question}
standalone question:""")

In [ ]:
qa = ConversationalRetrievalChain.from_llm(llm=llm, retriever=db.as_retriever(), condense_question_prompt=CONDENSE_QUESTION_PROMPT, return_source_documents=True, verbose=False)

In [ ]:
chat_history = []
query = """What are the different methods used in this paper?"""
result = qa({"question":query, "chat_history":chat_history})

In [ ]:
print(result['answer'])